# 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans 

# 2. Load  Neighborhood data and Geospacial data

In [2]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'lxml')
postalCodeList = []
boroughList = []
neighborhoodList = []

In [3]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [4]:
df = pd.DataFrame({"PostalCode": postalCodeList,"Borough": boroughList,"Neighborhood": neighborhoodList})
df_dropna = df[df.Borough != "Not assigned"].reset_index(drop=True)
df_grouped = df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ", ".join(x.astype(str))).reset_index()
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M2V,Not assigned,Not assigned
1,M4H,East York,Thorncliffe Park
2,M4S,Central Toronto,Davisville
3,M5Y,Not assigned,Not assigned
4,M4W,Downtown Toronto,Rosedale


In [6]:
coordinates = pd.read_csv("http://cocl.us/Geospatial_data")
coordinates.rename(columns={'Postal Code':'PostalCode'},inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# 3. Merge data together on the Postal Code

In [7]:
df_2 = df_grouped.merge(coordinates, on="PostalCode", how="left")
df_2.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [8]:
!pip install folium

# 4. Select data from the Borough contains 'Toronto'

In [9]:
T_geodata=df_2[df_2['Borough'].str.contains("Toronto")]
T_geodata

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(T_geodata['Borough'].unique()),
            T_geodata.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


In [11]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.653963, -79.387207.


# 5. Explore Toronto

In [12]:
CLIENT_ID = 'B0V3S4Z5LD3MRGEPKGZ5EIEPIJLLOW3VGILQXE10WJ2JZLZF' 
CLIENT_SECRET = 'MHUMWGFZGB0QY4IEOCG2XKSRPMR1Q42PEFSFLJA0OAWWBTXF' 
VERSION = '20180605' # Foursquare API version

In [15]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
T_nearby_venues = getNearbyVenues(names=T_geodata['Neighborhood'],
                                   latitudes=T_geodata['Latitude'],
                                   longitudes=T_geodata['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [17]:
T_nearby_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
2,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
3,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
5,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop
6,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
7,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
8,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
9,"The Danforth West, Riverdale",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant


In [18]:
T_nearby_venues.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Afghan Restaurant,1,1,1,1,1,1
Airport,1,1,1,1,1,1
Airport Food Court,1,1,1,1,1,1
Airport Lounge,2,2,2,2,2,2
Airport Service,3,3,3,3,3,3
Airport Terminal,2,2,2,2,2,2
American Restaurant,24,24,24,24,24,24
Antique Shop,2,2,2,2,2,2
Aquarium,5,5,5,5,5,5


In [19]:
T_rank = pd.get_dummies(T_nearby_venues[['Venue Category']], prefix="", prefix_sep="")
T_rank.drop(['Neighborhood'],axis=1,inplace=True) 
T_rank.insert(loc=0, column='Neighborhood', value=T_nearby_venues['Neighborhood'] )
T_rank

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
neighborhood_latitude = df_2.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_2.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_2.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

In [20]:
T_grouped = T_rank.groupby('Neighborhood').mean().reset_index()
T_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0000,0.0000,0.000,0.0000,0.000,0.02,0.0,0.0,...,0.0,0.020000,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.000000
1,Berczy Park,0.0,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.0,0.0,...,0.0,0.018182,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.058824
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0625,0.0625,0.125,0.1875,0.125,0.00,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000


In [21]:
results = requests.get('https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}').json()
results

{'meta': {'code': 400,
  'errorType': 'invalid_auth',
  'errorDetail': 'Missing access credentials. See https://developer.foursquare.com/docs/api/configuration/authentication for details.',
  'requestId': '5e568a82df2774001beb430b'},
 'response': {}}

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = T_grouped['Neighborhood']

for ind in np.arange(T_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(T_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Restaurant,Bar,Thai Restaurant
1,Berczy Park,Coffee Shop,Café,Bakery,Seafood Restaurant,Farmers Market
2,"Brockton, Exhibition Place, Parkdale Village",Café,Breakfast Spot,Coffee Shop,Performing Arts Venue,Furniture / Home Store
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Brewery,Skate Park,Spa
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boat or Ferry
5,"Cabbagetown, St. James Town",Coffee Shop,Pizza Place,Bakery,Pub,Chinese Restaurant
6,Central Bay Street,Coffee Shop,Italian Restaurant,Juice Bar,Japanese Restaurant,Burger Joint
7,"Chinatown, Grange Park, Kensington Market",Bar,Café,Chinese Restaurant,Coffee Shop,Vietnamese Restaurant
8,Christie,Grocery Store,Café,Park,Gas Station,Candy Store
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant


# 6. Clustering Toronto to find the best location to open a new coffee shop

In [24]:
from sklearn.cluster import KMeans 
k = 5
toronto_grouped_clustering = T_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [25]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_full = T_geodata

toronto_full = toronto_full.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_full

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Other Great Outdoors,Health Food Store,Pub,Coffee Shop,Trail
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Ice Cream Shop
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Brewery,Burrito Place,Sushi Restaurant,Ice Cream Shop,Movie Theater
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Brewery,Gastropub,American Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Bus Line,Yoga Studio,Distribution Center
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Breakfast Spot,Park,Department Store,Food & Drink Shop,Hotel
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Chinese Restaurant,Sporting Goods Shop
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Sandwich Place,Pizza Place,Dessert Shop,Sushi Restaurant,Café
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,1,Playground,Tennis Court,Park,Falafel Restaurant,Event Space
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,0,Coffee Shop,Pub,Bagel Shop,Restaurant,Supermarket


In [26]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,0,"Adelaide, King, Richmond",Coffee Shop,Café,Restaurant,Bar,Thai Restaurant
1,0,Berczy Park,Coffee Shop,Café,Bakery,Seafood Restaurant,Farmers Market
2,0,"Brockton, Exhibition Place, Parkdale Village",Café,Breakfast Spot,Coffee Shop,Performing Arts Venue,Furniture / Home Store
3,0,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Brewery,Skate Park,Spa
4,0,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boat or Ferry
5,0,"Cabbagetown, St. James Town",Coffee Shop,Pizza Place,Bakery,Pub,Chinese Restaurant
6,0,Central Bay Street,Coffee Shop,Italian Restaurant,Juice Bar,Japanese Restaurant,Burger Joint
7,0,"Chinatown, Grange Park, Kensington Market",Bar,Café,Chinese Restaurant,Coffee Shop,Vietnamese Restaurant
8,0,Christie,Grocery Store,Café,Park,Gas Station,Candy Store
9,0,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant


In [27]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = [] 
for lat, lon, neighbourhood, cluster in zip(T_geodata['Latitude'], T_geodata['Longitude'], T_nearby_venues['Neighborhood'],neighborhoods_venues_sorted['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [29]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 0, neighborhoods_venues_sorted.columns[[1] + list(range(5, neighborhoods_venues_sorted.shape[1]))]]

,Neighborhood,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Bar,Thai Restaurant
1,Berczy Park,Seafood Restaurant,Farmers Market
2,"Brockton, Exhibition Place, Parkdale Village",Performing Arts Venue,Furniture / Home Store
3,Business Reply Mail Processing Centre 969 Eastern,Skate Park,Spa
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Harbor / Marina,Boat or Ferry
5,"Cabbagetown, St. James Town",Pub,Chinese Restaurant
6,Central Bay Street,Japanese Restaurant,Burger Joint
7,"Chinatown, Grange Park, Kensington Market",Coffee Shop,Vietnamese Restaurant
8,Christie,Gas Station,Candy Store
9,Church and Wellesley,Gay Bar,Restaurant


In [31]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 1, neighborhoods_venues_sorted.columns[[1] + list(range(5, neighborhoods_venues_sorted.shape[1]))]]

,Neighborhood,4th Most Common Venue,5th Most Common Venue
24,"Moore Park, Summerhill East",Falafel Restaurant,Event Space
28,Rosedale,Dim Sum Restaurant,Falafel Restaurant


In [32]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 2, neighborhoods_venues_sorted.columns[[1] + list(range(5, neighborhoods_venues_sorted.shape[1]))]]

,Neighborhood,4th Most Common Venue,5th Most Common Venue
29,Roselawn,Farmers Market,Falafel Restaurant


In [33]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 3, neighborhoods_venues_sorted.columns[[1] + list(range(5, neighborhoods_venues_sorted.shape[1]))]]

,Neighborhood,4th Most Common Venue,5th Most Common Venue
17,"Forest Hill North, Forest Hill West",Bus Line,Yoga Studio


In [34]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 4, neighborhoods_venues_sorted.columns[[1] + list(range(5, neighborhoods_venues_sorted.shape[1]))]]

,Neighborhood,4th Most Common Venue,5th Most Common Venue
22,Lawrence Park,Yoga Studio,Distribution Center
